In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
traindf=pd.read_csv("drive/My Drive/Colab Notebooks/Final Project/Task 1/train_add3.csv")
traindf.head()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3, sharey=True, figsize=[15, 5])
# sns.distplot(traindf['number of horizontal rebars'], kde=False, color='r', ax=ax[0])
# sns.distplot(traindf['number of vertical rebars'], kde=False, color='r', ax=ax[1])
sns.distplot(traindf['Total rebars'], kde=False, color='r', ax=ax[2])

In [0]:
testdf=pd.read_csv("drive/My Drive/Colab Notebooks/Final Project/Task 1/testName.csv")
testdf.head()

In [0]:
import os
from keras.preprocessing.image import ImageDataGenerator

# base_dir = '.\\train_img'
# train_dir = os.path.join(base_dir, 'train_img')
train_dir = 'drive/My Drive/Colab Notebooks/Final Project/Task 1/totalDepthPic'
test_dir = 'drive/My Drive/Colab Notebooks/Final Project/Task 1/totalDepthPic'
BATCH_SIZE = 30

def Resample():
    datagen = ImageDataGenerator(
        # width_shift_range=10,
        # height_shift_range=10,
        # rotation_range=15,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # vertical_flip=True,
        rescale=1./255., 
        validation_split=0.25
    )
    train_generator = datagen.flow_from_dataframe( 
        dataframe=traindf, 
        directory=train_dir, 
        x_col='Pic Name', 
        y_col='Total rebars', 
        has_ext=False, 
        subset="training", 
        class_mode="raw", 
        batch_size=BATCH_SIZE,
        target_size=(150, 150)
    )

    validation_generator = datagen.flow_from_dataframe( 
        dataframe=traindf, 
        directory=train_dir,
        x_col='Pic Name', 
        y_col='Total rebars', 
        has_ext=False, 
        subset="validation", 
        class_mode="raw",
        batch_size=BATCH_SIZE,
        target_size=(150, 150)
        )

    return train_generator, validation_generator

test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=test_dir,
    x_col='Pic Name',
    target_size=(150, 150),
    color_mode='rgb',
    shuffle=False,
    class_mode=None
)

In [0]:
from keras.applications.resnet_v2 import ResNet50V2

base = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation=None))

In [0]:
from keras import models

model = models.load_model('drive/My Drive/Colab Notebooks/Final Project/Task 1/task1_top.h5', compile=False)

In [0]:
import keras.backend as K

def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [0]:
from keras import optimizers

model.compile(
    optimizer=optimizers.RMSprop(lr=2e-5),
    loss=RMSE,
    metrics=[RMSE]
)

In [0]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(
    filename="drive/My Drive/Colab Notebooks/Final Project/Task 1/yo_task1_resnet_unfrozen.log",
    separator=',',
    append=True
)

In [0]:
for num_resamp in range(2):
    train_generator, validation_generator = Resample()
    history = model.fit_generator(
        generator=train_generator,
        callbacks=[csv_logger],
        steps_per_epoch=50,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10
    )

In [0]:
import matplotlib.pyplot as plt

df_history = pd.read_csv('drive/My Drive/Colab Notebooks/Final Project/Task 1/yo_task1_resnet_unfrozen.log')
loss = df_history.loc[100:, 'loss']
val_loss = df_history.loc[100:, 'val_loss']

epochs = loss.index

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
pred = model.predict_generator(test_generator)
pred

In [0]:
# pred_int = pred.astype('int')
testdf['Total rebars'] = pred
testdf.head()

In [0]:
testdf.to_csv(
    "drive/My Drive/Colab Notebooks/Final Project/Task 1/Result/yoresult46.csv",
    index=False,
    encoding='utf-8'
)

In [0]:
model.save('drive/My Drive/Colab Notebooks/Final Project/Task 1/yomodel.h5')